# 1. Membuat Koneksi ke SQL Server

In [142]:
import pyodbc
import pandas as pd
from sqlalchemy import create_engine
print(pyodbc.drivers())

['SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'MySQL ODBC 8.0 ANSI Driver', 'MySQL ODBC 8.0 Unicode Driver', 'ODBC Driver 17 for SQL Server', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']


In [143]:
server = 'DESKTOP-OQ4U65F'
user = 'sa'
password = '12345678'


# DB Staging
conn_str_staging = (
    f"mssql+pyodbc://{user}:{password}@{server}/Staging"
    "?driver=ODBC+Driver+17+for+SQL+Server"
)

# DB DWH
conn_str_dwh = (
    f"mssql+pyodbc://{user}:{password}@{server}/DWH"
    "?driver=ODBC+Driver+17+for+SQL+Server"
)

# Buat engine
engine_staging = create_engine(conn_str_staging)
engine_dwh = create_engine(conn_str_dwh)

# 2. Ekstrak, Transform, dan Load data (ETL)

## 2.1 Branch -> DimBranch

In [144]:
# Ekstrak Dari staging
df_branch = pd.read_sql("SELECT * FROM branch", engine_staging)
df_branch.head()

,branch_id,branch_name,branch_location
0,1,KC Jakarta,Jl. Gatot Subroto No 13
1,2,KC Bogor,Jl. Padjajaran No 43
2,3,KC Depok,Jl. Raya Sawangan No 34
3,4,KC Tangerang,Jl. Cisauk No 50
4,5,KC Bekas,Jl. Ahmad Yani No 23


In [145]:
df_branch_dwh = pd.read_sql("SELECT * FROM DimBranch", engine_dwh)
print(df_branch_dwh.head())
print(df_branch_dwh.columns)

Empty DataFrame
Columns: [BranchID, BranchName, BranchLocation]
Index: []
Index(['BranchID', 'BranchName', 'BranchLocation'], dtype='object')


In [146]:
# Load ke DWH
df_branch.columns = ['BranchID', 'BranchName', 'BranchLocation']
df_branch.to_sql("DimBranch", engine_dwh, if_exists="append", index=False)

5

In [147]:
df_branch_dwh = pd.read_sql("SELECT * FROM DimBranch", engine_dwh)
df_branch_dwh.head()

,BranchID,BranchName,BranchLocation
0,1,KC Jakarta,Jl. Gatot Subroto No 13
1,2,KC Bogor,Jl. Padjajaran No 43
2,3,KC Depok,Jl. Raya Sawangan No 34
3,4,KC Tangerang,Jl. Cisauk No 50
4,5,KC Bekas,Jl. Ahmad Yani No 23


## 2.2 Customer -> DimCustomer

In [148]:
# Ekstrak Dari staging
df_customer = pd.read_sql("SELECT * FROM customer", engine_staging)
df_customer.head()

,customer_id,customer_name,address,city_id,age,gender,email
0,1,Shelly Juwita,Jl. Boulevard No. 31,2,25,female,shelly@gmail.com
1,2,Bobi Rinaldo,Jl. Mangga No. 1,3,31,male,Bobi@gmail.com
2,3,Adam Malik,Jl. Kincir Angin No. 50,5,23,male,Adam@gmail.com
3,4,Susi Rahmawati,Jl. Kenanga No. 11,7,45,female,Susi@gmail.com
4,5,Dimas Prasetyo,Jl. Niagara No. 69,8,32,male,Dimas@gmail.com


In [149]:
df_state= pd.read_sql("SELECT * FROM state", engine_staging)
df_state.head()

,state_id,state_name
0,1,Jakarta Utara
1,2,Jakarta Selatan
2,3,Jakarta Pusat
3,4,Jakarta Timur
4,5,Jakarta Barat


In [150]:
df_city = pd.read_sql("SELECT * FROM city", engine_staging)
df_city.head()

,city_id,city_name,state_id
0,1,Cilincing,1
1,2,Kelapa Gading,1
2,3,Tanjung Priok,1
3,4,Koja,1
4,5,Pademangan,1


In [151]:
df_customer_new= pd.read_sql("select cs.customer_id, cs.customer_name, cs.address, cm.city_name, cm.state_name, cs.age, cs.gender, cs.email from customer as cs left join (select city_id,city_name,ct.state_id,state_name from city as ct left join state as st on ct.state_id = st.state_id)as cm on cs.city_id = cm.city_id",engine_staging)
df_customer_new.head()

,customer_id,customer_name,address,city_name,state_name,age,gender,email
0,1,Shelly Juwita,Jl. Boulevard No. 31,Kelapa Gading,Jakarta Utara,25,female,shelly@gmail.com
1,2,Bobi Rinaldo,Jl. Mangga No. 1,Tanjung Priok,Jakarta Utara,31,male,Bobi@gmail.com
2,3,Adam Malik,Jl. Kincir Angin No. 50,Pademangan,Jakarta Utara,23,male,Adam@gmail.com
3,4,Susi Rahmawati,Jl. Kenanga No. 11,Cilandak,Jakarta Selatan,45,female,Susi@gmail.com
4,5,Dimas Prasetyo,Jl. Niagara No. 69,Jagakarsa,Jakarta Selatan,32,male,Dimas@gmail.com


In [152]:
df_DimCustomer = pd.read_sql("SELECT * FROM DimCustomer", engine_dwh)
df_DimCustomer.head()

,CustomerID,CustomerName,Address,CityName,StateName,Age,Gender,Email


In [153]:
df_DimCustomer.columns

Index(['CustomerID', 'CustomerName', 'Address', 'CityName', 'StateName', 'Age',
       'Gender', 'Email'],
      dtype='object')

In [154]:
# Load ke DWH
df_customer_new.columns = ['CustomerID', 'CustomerName', 'Address', 'CityName', 'StateName', 'Age','Gender', 'Email']
df_customer_new.to_sql("DimCustomer", engine_dwh, if_exists="append", index=False)

20

In [155]:
df_DimCustomer = pd.read_sql("SELECT * FROM DimCustomer", engine_dwh)
df_DimCustomer.head()

,CustomerID,CustomerName,Address,CityName,StateName,Age,Gender,Email
0,1,Shelly Juwita,Jl. Boulevard No. 31,Kelapa Gading,Jakarta Utara,25,female,shelly@gmail.com
1,2,Bobi Rinaldo,Jl. Mangga No. 1,Tanjung Priok,Jakarta Utara,31,male,Bobi@gmail.com
2,3,Adam Malik,Jl. Kincir Angin No. 50,Pademangan,Jakarta Utara,23,male,Adam@gmail.com
3,4,Susi Rahmawati,Jl. Kenanga No. 11,Cilandak,Jakarta Selatan,45,female,Susi@gmail.com
4,5,Dimas Prasetyo,Jl. Niagara No. 69,Jagakarsa,Jakarta Selatan,32,male,Dimas@gmail.com


## 2.3 Account -> DimAccount

In [156]:
df_account = pd.read_sql("SELECT * FROM account", engine_staging)
df_account.head()

,account_id,customer_id,account_type,balance,date_opened,status
0,1,1,saving,1500000,2020-05-01 09:00:00,active
1,2,2,saving,500000,2020-06-01 10:00:00,active
2,3,1,checking,25000000,2020-06-21 09:00:00,active
3,4,3,checking,4500000,2021-06-24 11:00:00,terminated
4,5,4,saving,75000000,2020-06-29 13:00:00,active


In [158]:
df_account_dwh = pd.read_sql("SELECT * FROM DimAccount", engine_dwh)
df_account_dwh.head()

,AccountID,CustomerID,AccountType,Balance,DateOpened,Status


In [159]:
# Load ke DWH
df_account.columns = ['AccountID', 'CustomerID', 'AccountType', 'Balance', 'DateOpened','Status']
df_account.to_sql("DimAccount", engine_dwh, if_exists="append", index=False)

23

In [160]:
df_account_dwh = pd.read_sql("SELECT * FROM DimAccount", engine_dwh)
df_account_dwh.head()

,AccountID,CustomerID,AccountType,Balance,DateOpened,Status
0,1,1,saving,1500000,2020-05-01 09:00:00,active
1,2,2,saving,500000,2020-06-01 10:00:00,active
2,3,1,checking,25000000,2020-06-21 09:00:00,active
3,4,3,checking,4500000,2021-06-24 11:00:00,terminated
4,5,4,saving,75000000,2020-06-29 13:00:00,active


## 2.4 Transaction -> Fact Transaction 

In [161]:
df_transaction_csv= pd.read_sql("SELECT * FROM transaction_csv", engine_staging)
df_transaction_csv.head()

,transaction_id,account_id,transaction_date,amount,transaction_type,branch_id
0,14,13,2024-01-21 14:00:00,1500000,Deposit,4
1,15,14,2024-01-21 08:00:00,500000,Transfer,3
2,16,15,2024-01-22 09:00:00,100000,Deposit,1
3,17,16,2024-01-22 13:10:00,100000,Withdrawal,5
4,18,17,2024-01-22 10:20:00,700000,Deposit,5


In [162]:
df_transaction_db= pd.read_sql("SELECT * FROM transaction_db", engine_staging)
df_transaction_db.head()

,transaction_id,account_id,transaction_date,amount,transaction_type,branch_id
0,1,1,2024-01-17 09:10:00,100000,Deposit,1
1,2,2,2024-01-17 10:10:00,1000000,Deposit,1
2,3,3,2024-01-18 08:30:00,10000000,Transfer,1
3,4,3,2024-01-18 10:45:00,1000000,Withdrawal,1
4,5,5,2024-01-18 11:10:00,200000,Deposit,1


In [163]:
df_transaction_excel= pd.read_sql("SELECT * FROM transaction_excel", engine_staging)
df_transaction_excel.head()

,transaction_id,account_id,transaction_date,amount,transaction_type,branch_id
0,6,6,2024-01-18 13:10:00,50000,Withdrawal,1
1,7,6,2024-01-19 14:00:00,100000,Payment,1
2,11,10,2024-01-20 15:00:00,1000000,Transfer,1
3,12,11,2024-01-20 10:00:00,500000,Deposit,1
4,13,12,2024-01-20 12:10:10,500000,Withdrawal,5


In [171]:
df_facttransaction = pd.read_sql("SELECT * FROM FactTransaction", engine_dwh)
df_facttransaction.head()

,TransactionID,AccountID,TransactionDate,Amount,TransactionType,BranchID


In [166]:
df_facttransaction.columns

Index(['TransactionID', 'AccountID', 'TransactionDate', 'Amount',
       'TransactionType', 'BranchID'],
      dtype='object')

In [170]:
df_transaction_merged = pd.read_sql("SELECT * FROM transaction_db UNION SELECT * FROM transaction_excel UNION SELECT * FROM transaction_csv",engine_staging)
df_transaction_merged.head()

,transaction_id,account_id,transaction_date,amount,transaction_type,branch_id
0,1,1,2024-01-17 09:10:00,100000,Deposit,1
1,2,2,2024-01-17 10:10:00,1000000,Deposit,1
2,3,3,2024-01-18 08:30:00,10000000,Transfer,1
3,4,3,2024-01-18 10:45:00,1000000,Withdrawal,1
4,5,5,2024-01-18 11:10:00,200000,Deposit,1


In [172]:
# Load ke DWH ['A
df_transaction_merged.columns = ['TransactionID', 'AccountID', 'TransactionDate', 'Amount','TransactionType', 'BranchID']
df_transaction_merged.to_sql("FactTransaction", engine_dwh, if_exists="append", index=False)

25

In [173]:
df_facttransaction = pd.read_sql("SELECT * FROM FactTransaction", engine_dwh)
df_facttransaction.head()

,TransactionID,AccountID,TransactionDate,Amount,TransactionType,BranchID
0,1,1,2024-01-17 09:10:00,100000,Deposit,1
1,2,2,2024-01-17 10:10:00,1000000,Deposit,1
2,3,3,2024-01-18 08:30:00,10000000,Transfer,1
3,4,3,2024-01-18 10:45:00,1000000,Withdrawal,1
4,5,5,2024-01-18 11:10:00,200000,Deposit,1


In [140]:
engine_staging.dispose()
engine_dwh.dispose()